In [13]:
import numpy as np
import pandas as pd
import warnings
from sklearn.feature_selection import SelectKBest
warnings.filterwarnings('ignore')

In [8]:
df = pd.read_csv('C:/Data/TestingData/aligned_reduced.csv')
df

,x1,y1,xVel1,yVel1,xA1,yA1,xS1,yS1,xC1,yC1,...,yVel200,xA200,yA200,xS200,yS200,xC200,yC200,nAC200,nS200,Class
0,-1414.14,-535.22,-17.88,-7.23,0.0,0.0,0.00,0.00,0.00,0.00,...,-16.85,0.00,0.00,0.0,0.00,0.00,0.00,29,0,0
1,-1406.36,698.39,-16.54,-6.95,-1.0,0.0,-944.07,-396.62,0.00,0.00,...,-12.61,0.00,-1.00,0.0,0.00,0.00,0.00,13,0,0
2,-1404.81,939.49,-5.44,-2.20,0.0,0.0,0.00,0.00,0.00,0.00,...,-10.69,0.00,0.00,0.0,0.00,0.00,0.00,27,1,0
3,-1403.15,-350.53,-6.39,0.13,0.0,0.0,0.00,0.00,0.00,0.00,...,0.63,-0.56,0.06,0.0,0.00,0.16,-0.28,30,0,1
4,-1402.23,259.25,-6.22,14.16,-1.0,0.0,-20.01,60.79,0.00,0.00,...,-9.77,-1.00,1.00,4.0,-0.04,0.00,0.00,18,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999,1398.12,-256.12,3.98,-6.42,0.0,0.0,0.00,0.00,0.00,0.00,...,-5.92,-0.13,-0.29,0.0,0.00,-0.12,-0.30,5,0,0
6000,1399.16,954.83,4.54,-6.29,0.0,0.0,0.00,0.00,-0.13,-0.29,...,-4.87,0.00,0.00,0.0,0.00,-0.26,-0.18,12,0,0
6001,1400.87,-235.20,9.33,-10.79,0.0,0.0,0.00,0.00,0.00,0.00,...,-0.80,0.00,0.00,0.0,0.00,0.00,0.00,5,0,0
6002,1402.68,-5.93,6.63,6.09,0.0,0.0,0.00,0.00,0.00,0.00,...,0.57,0.00,0.00,0.0,0.00,0.00,0.00,1,0,0


In [9]:
Y = np.asarray(df.Class)
X = np.asarray(df.drop(columns=['Class'],axis=1))


# Filter Method

In [20]:
Kbest = SelectKBest(k=500)
T = Kbest.fit(X,Y)
X_new = T.transform(X)
X_new

array([[ -7.23,   0.  ,   0.  , ...,   0.  , -16.85,   0.  ],
       [ -6.95,   0.  ,  -1.  , ...,  -1.  , -12.61,  -1.  ],
       [ -2.2 ,   0.  ,   0.  , ...,   0.  , -10.69,   0.  ],
       ...,
       [-10.79,   0.  ,   0.  , ...,   0.  ,  -0.8 ,   0.  ],
       [  6.09,   0.  ,   0.  , ...,   0.  ,   0.57,   0.  ],
       [ 14.33,   1.  ,   1.  , ...,   1.  ,   5.2 ,   1.  ]])

# Wrapper Method

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.feature_selection import RFE
# from sklearn.feature_selection import RFECV
import pickle as pkl

In [26]:
M = LogisticRegression()
# M = LogisticRegressionCV()
T = RFE(M,n_features_to_select=200)
# T = RFECV(M,cv=5)
T.fit(X_new,Y)
pkl.dump(T,open('T.p','wb'))

In [27]:
T = pkl.load(open('T.p','rb'))

In [33]:
x_200 = T.transform(X_new)
print(T)

RFE(estimator=LogisticRegression(), n_features_to_select=200)


# Embedded Method

In [35]:
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC

In [36]:
M = LinearSVC(C=0.01,penalty='l1',dual=False)
M.fit(X,Y)

LinearSVC(C=0.01, dual=False, penalty='l1')

In [39]:
T = SelectFromModel(M,prefit=True)
T.transform(X)

array([[-1414.14,  -535.22,  -589.94, ...,  -473.61,   687.57,  -810.85],
       [-1406.36,   698.39,  -885.04, ...,   213.81,   571.48,   605.72],
       [-1404.81,   939.49,  -694.49, ...,   808.88,  1317.73,   241.66],
       ...,
       [ 1400.87,  -235.2 , -1170.04, ...,   365.4 ,   542.03,  -749.83],
       [ 1402.68,    -5.93,   926.42, ...,  -657.16,  1034.42,   -81.59],
       [ 1403.72,   133.09,  -695.11, ...,  -802.1 ,  1187.52,   259.33]])

# Correlation Filter

In [52]:
c = np.asarray([])
for i in range(X.shape[1]):
    # c = np.append(c,np.corrcoef(X[:,i],Y)[0,1])
    fi = X[:,i]
    c = np.append(c,fi.dot(Y)/((Y.dot(Y)*fi.dot(fi))**0.5))

In [58]:
idx = np.argsort(c)[::-1]
col = df.columns[np.abs(idx)][:10]
col

Index(['yA115', 'yA102', 'yA41', 'yA59', 'yA179', 'yA155', 'yA94', 'yA6',
       'yA197', 'yA47'],
      dtype='object')

In [63]:
# we can use one lin
idx = np.argsort(X.T.dot(Y))[::-1][:10]
col = df.columns[idx]
col


Index(['x198', 'x185', 'x84', 'x115', 'x158', 'x150', 'x169', 'x6', 'x162',
       'x96'],
      dtype='object')

In [65]:
D= X.T.dot(Y)
len(D)

2400